In [ ]:
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import SparkSession

import datetime
import pyspark.sql.functions as F 
import dateutil.relativedelta

spark = SparkSession.builder \
            .master("yarn") \
            .appName("akk_2") \
            .config("spark.executor.memory", "5530m")\
            .config("spark.executor.instances", 4)\
            .config("spark.executor.cores", 2)\
            .config("spark.files.overwrite",True)\
            .getOrCreate()

In [ ]:
tags_verified = spark.read.parquet('/user/master/data/snapshots/tags_verified/actual')

In [ ]:


# def input_paths(date,depth):
#     range_ = []
#     date = datetime.datetime.strptime(date, "%Y-%m-%d")
#     i = 0
#     while i < depth:
#         date_2 = date - dateutil.relativedelta.relativedelta(days=i)
#         path_ = f'/user/andrew_0/data/events/date={date_2.strftime("%Y-%m-%d")}/event_type=message'
#         range_.append(path_)
#         i = i+1
#     return(range_)


# range_ = input_paths(date_from,depth)

# df = spark.read.parquet(*range_)

# df_explode = df.withColumn("tag", F.explode(df.event.tags)).where("event.message_channel_to is not null")
# df_explode = df_explode.select(df_explode.event.message_from.alias("message_from"),df_explode.tag)

# df_explode = df_explode.groupBy('tag') \
#                       .agg(F.countDistinct('message_from').alias("suggested_count")) \
#                       .filter(F.col('suggested_count') >= 100) \
#                       .orderBy(F.col("suggested_count").desc()) \
#                       .join(tags_verified, ['tag'], how='left_anti')

# df_explode.write.format('parquet').save('/user/andrew0/data/analytics/candidates_d84_pyspark',mode='overwrite')

# df_7 = spark.read.parquet('/user/andrew0/data/analytics/candidates_d7_pyspark')
# df_7.describe().show()

# df_84 = spark.read.parquet('/user/andrew0/data/analytics/candidates_d84_pyspark')
# df_84.describe().show()



# def input_paths(date, depth):
#     dt = datetime.datetime.strptime(date, '%Y-%m-%d')
#     return [f"/user/username/data/events/date={(dt-datetime.timedelta(days=x)).strftime('%Y-%m-%d')}/event_type=message" for x in range(depth)]

# paths = input_paths('2022-05-31', M)
# messages = spark.read.parquet(*paths)
# all_tags = messages.where("event.message_channel_to is not null").selectExpr(["event.message_from as user", "explode(event.tags) as tag"]).groupBy("tag").agg(F.expr("count(distinct user) as suggested_count")).where("suggested_count >= 100")
# verified_tags = spark.read.parquet("/user/master/data/snapshots/tags_verified/actual")
# candidates = all_tags.join(verified_tags, "tag", "left_anti")

# candidates.write.parquet(f'/user/username/data/analytics/candidates_d{M}_pyspark')

# /usr/lib/spark/bin/spark-submit --master yarn --deploy-mode cluster verified_tags_candidates.py 2022-05-31 5 300 

# /user/andrew0/5.2.4/analytics/verified_tags_candidates_d5/date=2022-05-31

# date_from = '2022-05-31'
# depth = 5
# sugested_users = 300
# input_path = '/user/andrew_0/data/events'
# tags_verified_path = '/user/master/data/snapshots/tags_verified/actual'
# output_path = '/user/andrew0/5.2.4/analytics/verified_tags_candidates_d5/'+'date='+date_from

 



In [ ]:
# interests_d7_eth =  spark.read.parquet('/user/examples/data/interests_d7')
# interests_d7_eth.show(10,False)
# interests_d28_eth = spark.read.parquet('/user/examples/data/interests_d28')
# interests_d28_eth.show(10,False)

In [15]:
date_from = '2022-06-04'
depth = 5
# sugested_users = 300
input_path = '/user/andrew_0/data/events'
# tags_verified_path = '/user/master/data/snapshots/tags_verified/actual'
# output_path = '/user/andrew0/5.2.4/analytics/verified_tags_candidates_d5/'+'date='+date_from

In [ ]:
# range_

In [11]:
def input_paths(date,depth):
    range_ = []
    date = datetime.datetime.strptime(date, "%Y-%m-%d")
    i = 0
    while i < depth:
        date_2 = date - dateutil.relativedelta.relativedelta(days=i)
        path_ = f'{input_path}/date={date_2.strftime("%Y-%m-%d")}/event_type=message'
        range_.append(path_)
        i = i+1
    return(range_)

In [ ]:
range_ = input_paths(date_from,depth)

df = spark.read.parquet(*range_)


In [ ]:
from pyspark.sql.window import Window

window_  = Window.partitionBy(["user"]).orderBy(F.col("suggested_count").desc(),F.col("tag").desc())

In [ ]:
def tag_tops(date_from,depth,spark):
    
    range_ = input_paths(date_from,depth)
    
    df = spark.read.parquet(*range_)
    
    window_  = Window.partitionBy(["user"]).orderBy(F.col("suggested_count").desc(),F.col("tag").desc())
    
    
    df = df.where("event.message_channel_to is not null")\
    .select([F.col("event.message_from").alias("user"), F.explode(F.col("event.tags")).alias("tag")])\
    .groupBy(["user","tag"]).agg(F.count("tag").alias("suggested_count"))\
    .withColumn("r_n",F.row_number().over(window_))\
    .withColumn("tag_top_1",F.lead("tag",0).over(window_))\
    .withColumn("tag_top_2",F.lead("tag",1).over(window_))\
    .withColumn("tag_top_3",F.lead("tag",2).over(window_))\
    .where("r_n = 1")\
    .select([F.col('user').alias('user_id'),F.col('tag_top_1'),F.col('tag_top_2'),F.col('tag_top_3')])
    
    print(date_from,depth)
    return df

In [ ]:
tag_tops('2022-06-04', 5, spark).repartition(1).write.parquet('/user/andrew0/data/tmp/tag_tops_06_04_5')

tag_tops('2022-05-04', 5, spark).repartition(1).write.parquet('/user/andrew0/data/tmp/tag_tops_05_04_5')

tag_tops('2022-05-04', 1, spark).repartition(1).write.parquet('/user/andrew0/data/tmp/tag_tops_05_04_1')

In [ ]:
spark.read.parquet('/user/andrew0/data/tmp/tag_tops_06_04_5').show(10,False)

### reaction

In [108]:
date_from = '2022-04-04'
depth = 1
input_path = '/user/andrew_0/data/events'

In [109]:
def input_paths(date,depth,event_type):
    range_ = []
    date = datetime.datetime.strptime(date, "%Y-%m-%d")
    i = 0
    while i < depth:
        date_2 = date - dateutil.relativedelta.relativedelta(days=i)
        path_ = f'{input_path}/date={date_2.strftime("%Y-%m-%d")}/event_type={event_type}'
        range_.append(path_)
        i = i+1
    return(range_)

In [127]:
# def tag_tops(date_from,depth,spark):
    
range_message = input_paths(date_from,depth,'message')
range_reaction = input_paths(date_from,depth,'reaction')

df_message = spark.read.parquet(*range_message)
df_message = df_message.select(F.col("event.*")).where("tags is not null")
df_message = df_message.select([F.col('message_id'),F.explode(F.col("tags")).alias("tag")])

df_reaction = spark.read.parquet(*range_reaction)
df_reaction = df_reaction.select(F.col("event.*")).where("reaction_type is not null")
df_reaction = df_reaction.select([F.col('message_id'),F.col("reaction_from").alias("user"),F.col("reaction_type")])

df = df_message.join(df_reaction,df_message.message_id ==  df_reaction.message_id,"inner").drop(df_reaction.message_id)

df_message.printSchema()
df_reaction.printSchema()
df.printSchema()

root
 |-- message_id: long (nullable = true)
 |-- tag: string (nullable = true)

root
 |-- message_id: long (nullable = true)
 |-- user: string (nullable = true)
 |-- reaction_type: string (nullable = true)

root
 |-- message_id: long (nullable = true)
 |-- tag: string (nullable = true)
 |-- user: string (nullable = true)
 |-- reaction_type: string (nullable = true)



In [137]:
from pyspark.sql.window import Window

window_  = Window.partitionBy(["user"]).orderBy(F.col("reaction_count").desc(),F.col("tag").desc())

In [129]:
df.show(10,False)

+----------+---+------+-------------+
|message_id|tag|user  |reaction_type|
+----------+---+------+-------------+
|1106403   |PHP|117765|dislike      |
|1106403   |PHP|76913 |like         |
|1106403   |PHP|136616|like         |
|1106403   |PHP|73185 |like         |
|1106403   |PHP|121920|dislike      |
|1106403   |PHP|8116  |like         |
|1106403   |PHP|101751|like         |
|1106403   |PHP|49172 |like         |
|1106403   |CSS|117765|dislike      |
|1106403   |CSS|76913 |like         |
+----------+---+------+-------------+
only showing top 10 rows



In [145]:
df.where("user = 136082")\
.groupBy(["user","tag","reaction_type"]).agg(F.count("message_id").alias("reaction_count"))\
.withColumn("r_n",F.row_number().over(window_))\
.withColumn("like_tag_top_1",F.lead("tag",0).over(window_))\
.withColumn("like_tag_top_2",F.lead("tag",1).over(window_))\
.show(10,False)
# .withColumn("tag_top_2",F.lead("tag",1).over(window_))\
# .withColumn("tag_top_3",F.lead("tag",2).over(window_))\
# .where("r_n = 1")\
# .select([F.col('user').alias('user_id'),F.col('tag_top_1'),F.col('tag_top_2'),F.col('tag_top_3')])

+------+-----------------+-------------+--------------+---+-----------------+-----------------+
|user  |tag              |reaction_type|reaction_count|r_n|like_tag_top_1   |like_tag_top_2   |
+------+-----------------+-------------+--------------+---+-----------------+-----------------+
|136082|Физика           |like         |2             |1  |Физика           |Научно-популярное|
|136082|Научно-популярное|like         |2             |2  |Научно-популярное|Лазеры           |
|136082|Лазеры           |like         |1             |3  |Лазеры           |Космонавтика     |
|136082|Космонавтика     |like         |1             |4  |Космонавтика     |votizen          |
|136082|votizen          |like         |1             |5  |votizen          |techmeme         |
|136082|techmeme         |like         |1             |6  |techmeme         |memeorandum      |
|136082|memeorandum      |like         |1             |7  |memeorandum      |democracy        |
|136082|democracy        |like         |